In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches

EXPORT_DIR = Path("../reports/exports"); EXPORT_DIR.mkdir(parents=True, exist_ok=True)

plt.rcParams.update({
    "figure.figsize": (10, 4.8),
    "axes.titlesize": 16,
    "axes.labelsize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
})


In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyArrowPatch

EXPORT_DIR = Path("../reports/exports"); EXPORT_DIR.mkdir(parents=True, exist_ok=True)


COL_BG   = "#FFFFFF"
COL_TEXT = "#1C2333"
COL_ACC  = "#0F62FE"   
COL_ACC2 = "#08BDBA"   
COL_BOX  = "#EFF4FF"   
COL_EDGE = "#1A3A7C"   
COL_SHD  = "#95A3B3"   

plt.rcParams.update({
    "figure.figsize": (12, 4.8),
    "axes.titlesize": 17,
    "axes.labelsize": 12,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "font.family": "DejaVu Sans",
})

def draw_shadow(ax, xy, w, h, r=12, dx=0.01, dy=-0.01, alpha=0.18):
    sh = patches.FancyBboxPatch(
        (xy[0]+dx, xy[1]+dy), w, h,
        boxstyle=patches.BoxStyle("Round", rounding_size=r),
        linewidth=0, facecolor=COL_SHD, alpha=alpha, zorder=0.5
    )
    ax.add_patch(sh)

def draw_box(ax, xy, w, h, title, lines, bold=False):
    
    draw_shadow(ax, xy, w, h)
    
    box = patches.FancyBboxPatch(
        xy, w, h,
        boxstyle=patches.BoxStyle("Round", rounding_size=12),
        linewidth=1.8, edgecolor=COL_EDGE, facecolor=COL_BOX, zorder=1
    )
    ax.add_patch(box)
    
    ax.text(
        xy[0]+w/2, xy[1]+h-0.08, title,
        ha="center", va="top", color=COL_TEXT,
        fontsize=13.5, fontweight="bold" if bold else "semibold"
    )
    
    text = "\n".join(lines)
    ax.text(
        xy[0]+w/2, xy[1]+h/2-0.05, text,
        ha="center", va="center", color=COL_TEXT, fontsize=11
    )

def arrow(ax, xy1, xy2):
    arr = FancyArrowPatch(
        xy1, xy2, arrowstyle="-|>", mutation_scale=14,
        linewidth=2.0, color=COL_ACC, connectionstyle="arc3,rad=0.0"
    )
    ax.add_patch(arr)

fig, ax = plt.subplots()
ax.set_facecolor(COL_BG); ax.axis("off")
ax.set_xlim(0, 1); ax.set_ylim(0, 1)


blocks = [
    ((0.03, 0.34), 0.18, 0.36,
     "Simulation Data",
     ["IllustrisTNG100", "Snapshots 18–33", "MPB extraction", "≥90% coverage (N≈2500)"], True),
    ((0.26, 0.34), 0.18, 0.36,
     "Preprocessing",
     ["Select features", "Standardize (z-score)", "QC & coverage checks"], False),
    ((0.49, 0.34), 0.18, 0.36,
     "Data Split",
     ["Train / Val / Test", "by subhalo ID"], False),
    ((0.72, 0.34), 0.18, 0.36,
     "Modeling",
     ["LSTM forecaster", "Horizons Δ = {1,3,5}"], False),
    ((0.72, 0.05), 0.25, 0.22,
     "Evaluation & Explainability",
     ["RMSE per horizon/feature", "Parity plots", "Baselines (persistence, ridge)", "Permutation importance", "±1σ sensitivity"], False),
]

for (xy,w,h,title,lines,bold) in blocks:
    draw_box(ax, xy, w, h, title, lines, bold=bold)


arrow(ax, (0.21, 0.52), (0.26, 0.52))
arrow(ax, (0.44, 0.52), (0.49, 0.52))
arrow(ax, (0.67, 0.52), (0.72, 0.52))

arrow(ax, (0.81, 0.34), (0.845, 0.27))

ax.set_title("End-to-End Pipeline for Forecasting Black Hole Evolution", color=COL_TEXT, pad=8)
plt.tight_layout()
fig.savefig(EXPORT_DIR/"pipeline_schematic.png", dpi=300, bbox_inches="tight")
fig.savefig(EXPORT_DIR/"pipeline_schematic.svg", bbox_inches="tight")
plt.close(fig)

print("Saved:", (EXPORT_DIR/"pipeline_schematic.png").as_posix())


Saved: ../reports/exports/pipeline_schematic.png


In [3]:
# === Pretty LSTM Architecture Schematic (publication style) ===
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyArrowPatch

COL_TEXT = "#1C2333"
COL_CELL = "#E8F5E9"   # soft green
COL_CELL_E = "#1B5E20" # dark green
COL_IN    = "#F4F6F8"  # light gray
COL_HEAD  = "#FFF3E0"  # soft orange
COL_OUT   = "#E3F2FD"  # light blue
COL_ACC   = "#0F62FE"  # blue
COL_SHD   = "#95A3B3"

def shadow(ax, xy, w, h, r=10, dx=0.01, dy=-0.01, a=0.18):
    ax.add_patch(patches.FancyBboxPatch(
        (xy[0]+dx, xy[1]+dy), w, h,
        boxstyle=patches.BoxStyle("Round", rounding_size=r),
        linewidth=0, facecolor=COL_SHD, alpha=a, zorder=0.5
    ))

def box(ax, xy, w, h, text, fc, ec, bold=False, fs=11):
    shadow(ax, xy, w, h)
    ax.add_patch(patches.FancyBboxPatch(
        xy, w, h, boxstyle=patches.BoxStyle("Round", rounding_size=10),
        linewidth=1.6, edgecolor=ec, facecolor=fc, zorder=1
    ))
    ax.text(xy[0]+w/2, xy[1]+h/2, text, ha="center", va="center",
            fontsize=fs, color=COL_TEXT, fontweight=("bold" if bold else "normal"), zorder=2)

def conn(ax, p1, p2, rad=0.0, color=COL_ACC, lw=2.0, ms=14):
    ax.add_patch(FancyArrowPatch(p1, p2, arrowstyle="-|>", mutation_scale=ms,
                                 linewidth=lw, color=color, connectionstyle=f"arc3,rad={rad}"))

fig, ax = plt.subplots(figsize=(12, 4.8))
ax.axis("off"); ax.set_xlim(0,1); ax.set_ylim(0,1)

T = 8
x0, y_in, y_cell = 0.07, 0.28, 0.56
dx = 0.085
w, h = 0.07, 0.16

# Input sequence boxes (x[1..T])
for i in range(T):
    box(ax, (x0+i*dx, y_in), w, h, f"x[{i+1}]", COL_IN, "#65737E", fs=10)

# LSTM cells
for i in range(T):
    box(ax, (x0+i*dx, y_cell), w, h, "LSTM", COL_CELL, COL_CELL_E, bold=True, fs=11)
    conn(ax, (x0+i*dx + w/2, y_in + h), (x0+i*dx + w/2, y_cell), rad=0.0, lw=1.8, ms=12)
    if i < T-1:
        conn(ax, (x0+i*dx + w, y_cell + h/2), (x0+(i+1)*dx, y_cell + h/2), rad=0.0, lw=2.2, ms=14)

# Dense head
x_head = x0 + (T-1)*dx + 0.12
box(ax, (x_head, y_cell), 0.14, h, "Dense Head", COL_HEAD, "#BF360C", bold=True)

conn(ax, (x0+(T-1)*dx + w, y_cell + h/2), (x_head, y_cell + h/2), lw=2.2, ms=14)

# Multi-horizon outputs
outs = [(0.18, "H=1"), (0.05, "H=3"), (-0.08, "H=5")]
for k, (off, label) in enumerate(outs):
    y = y_cell + off
    box(ax, (x_head + 0.18, y - 0.06), 0.18, 0.12, f"Predicted features\n({label})", COL_OUT, "#0D47A1", fs=10)
    conn(ax, (x_head + 0.14, y_cell + h/2), (x_head + 0.18, y), rad=0.0, lw=2.0, ms=12)

# Input feature legend (compact)
in_feats = ["BH Mass", "BH Accretion", "Stellar Mass", "SFR", "Halo Mass", "Velocity Dispersion"]
ax.text(0.5, 0.15, "Input vector at each timestep (z-scored):  " + ", ".join(in_feats),
        ha="center", va="center", fontsize=11, color=COL_TEXT)

ax.set_title("LSTM Architecture for Multi-Horizon Forecasting", pad=6, color=COL_TEXT)
plt.tight_layout()
fig.savefig(EXPORT_DIR/"lstm_architecture.png", dpi=300, bbox_inches="tight")
fig.savefig(EXPORT_DIR/"lstm_architecture.svg", bbox_inches="tight")
plt.close(fig)

print("Saved:", (EXPORT_DIR/"lstm_architecture.png").as_posix())


Saved: ../reports/exports/lstm_architecture.png


In [5]:
pipeline_caption = (
    "Overview of the end-to-end machine learning pipeline used to forecast black hole evolution. "
    "We extract most-bound-progenitor (MPB) tracks from IllustrisTNG100 across snapshots 18–33 and retain systems "
    "with ≥90% snapshot coverage (N≈2500). Tracks are standardized and split by subhalo ID into train/validation/test. "
    "An LSTM forecaster predicts future properties at horizons Δ={1,3,5} snapshots. Evaluation includes RMSE per horizon "
    "and feature, parity plots, baselines (persistence, ridge), and explainability analyses."
)

model_caption = (
    "Schematic of the LSTM forecaster. At each timestep, the input vector comprises z-scored physical properties "
    f"({', '.join(in_feats)}). The final hidden state feeds a dense head that outputs multi-horizon predictions "
    "for Δ={1,3,5}. We train with MSE loss and AdamW, using ReduceLROnPlateau and early stopping."
)

(Path(EXPORT_DIR/"caption_pipeline.txt")).write_text(pipeline_caption)
(Path(EXPORT_DIR/"caption_lstm.txt")).write_text(model_caption)
"Saved captions to exports/."


'Saved captions to exports/.'